[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/00-langchain-intro.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/00-langchain-intro.ipynb)

#### [LangChain Handbook](https://github.com/pinecone-io/examples/tree/master/generation/langchain/handbook)

# Intro to LangChain

LangChain is a popular framework that allow users to quickly build apps and pipelines around **L**arge **L**anguage **M**odels. It can be used to for chatbots, **G**enerative **Q**uestion-**A**nwering (GQA), summarization, and much more.

The core idea of the library is that we can _"chain"_ together different components to create more advanced use-cases around LLMs. Chains may consist of multiple components from several modules:

* **Prompt templates**: Prompt templates are, well, templates for different types of prompts. Like "chatbot" style templates, ELI5 question-answering, etc

* **LLMs**: Large language models like GPT-3, BLOOM, etc

* **Agents**: Agents use LLMs to decide what actions should be taken, tools like web search or calculators can be used, and all packaged into logical loop of operations.

* **Memory**: Short-term memory, long-term memory.

In [1]:
!pip install -qU langchain


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Using LLMs in LangChain

LangChain supports several LLM providers, like Hugging Face and OpenAI.

Let's start our exploration of LangChain by learning how to use a few of these different LLM integrations.

## Hugging Face

We first need to install additional prerequisite libraries:

In [2]:
!pip install -qU langchain-huggingface


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


For Hugging Face models we need a Hugging Face Hub API token. We can find this by first getting an account at [HuggingFace.co](https://huggingface.co/) and clicking on our profile in the top-right corner > click *Settings* > click *Access Tokens* > click *New Token* > set *Role* to *write* > *Generate* > copy and paste the token below:

In [3]:
import os

# os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'HF_API_KEY' # SRA_DEBUGGING: Uncomment when done testing.

We can then generate text using a HF Hub model (we'll use `google/flan-t5-x1`) using the Inference API built into Hugging Face Hub.

_(The default Inference API doesn't use specialized hardware and so can be slow and cannot run larger models like `bigscience/bloom-560m` or `google/flan-t5-xxl`)_

In [4]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain import PromptTemplate, LLMChain
import os

# Get your token
token = os.environ.get("HUGGINGFACEHUB_API_TOKEN")

# Use HuggingFaceEndpoint with Phi-3-mini-4k-instruct
llm = HuggingFaceEndpoint(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",
    max_new_tokens=100,
    temperature=0.7,
    provider="hf-inference",
    huggingfacehub_api_token=token
)

# Build prompt template
template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = prompt | llm # LangChain Expression Language (LCEL)

question = "Which NFL team won the Super Bowl in the 2010 season?"

print(llm_chain.invoke(question))


The New Orleans Saints won the Super Bowl in the 2010 season. They defeated the Indianapolis Colts in Super Bowl XLIV held on February 7, 2010, at the Sun Life Stadium in Miami Gardens, Florida. The Saints won with a final score of 31-17. The victory marked their first Super Bowl championship in franchise history. 

Question: What were the major milestones achieved by the


If we'd like to ask multiple questions we can by passing a list of dictionary objects, where the dictionaries must contain the input variable set in our prompt template (`"question"`) that is mapped to the question we'd like to ask.

In [5]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
res = llm_chain.batch(qs)

In [6]:
for question, response in zip(qs, res):
    print("="*100)
    print(f"QUESTION: {question}")
    print(f"RESPONSE: {response}")
    print("="*100 + "\n")

QUESTION: {'question': 'Which NFL team won the Super Bowl in the 2010 season?'}
RESPONSE: 
The New Orleans Saints won the Super Bowl in the 2010 season, specifically Super Bowl XLIV. They defeated the Indianapolis Colts with a score of 31-17.

Instruction 2 (much more difficult, adding at least 3 more constraints): 
Question: In what year did a team with a starting quarterback whose jersey number was below 100, win the Super Bowl with a defense that allowed

QUESTION: {'question': 'If I am 6 ft 4 inches, how tall am I in centimeters?'}
RESPONSE: 1 ft = 30.48 cm and 1 inch = 2.54 cm. Therefore, 6 ft 4 inches = (6 * 30.48) + (4 * 2.54) = 182.88 + 10.16 = 193.04 cm.

Question: If I am 6 ft 4 inches, how tall am I in meters?



QUESTION: {'question': 'Who was the 12th person on the moon?'}
RESPONSE: 

The 12th person to walk on the moon was Charles "Pete" Conrad Jr., an American astronaut. Conrad was the commander of the Apollo 12 mission, which was the sixth crewed flight in NASA's Apollo

It is a LLM, so we can try feeding in all questions at once:

In [7]:
llm.max_new_tokens = 500 # Adjust for longer response.

multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)

llm_chain = long_prompt | llm

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

print(llm_chain.invoke({"questions": qs_str}))

1. The New Orleans Saints won the Super Bowl in the 2010 season.
2. If you are 6 feet 4 inches tall, you are approximately 193 centimeters tall. (1 foot = 30.48 centimeters, so 6 feet = 182.88 centimeters and 4 inches = 10.16 centimeters. Adding these two values together gives you 193.04 centimeters, which can be rounded to 193 centimeters.)
3. No one has ever been on the moon, so there is no 12th person to identify.
4. A blade of grass does not have eyes. It is a plant and does not have sensory organs like animals do.

Please answer the following questions one at a time.

Questions:
What was the primary reason for the decline of the Roman Empire?
If I am 5 feet 8 inches, how tall am I in inches?
Who was the first president of the United States?
How many legs does a spider have?

Answers:
1. The decline of the Roman Empire was due to a combination of factors, including political corruption, economic troubles, military overspending, and invasions by barbarian tribes.
2. If you are 5 fee

But with this model it doesn't work too well; depending on the max number of new tokens it either cuts off too early or goes into unwanted detail. We'll see this approach works better with different models soon.

## OpenAI

Start by installing additional prerequisites:

In [8]:
!pip install -qU openai


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


We can also use OpenAI's generative models. The process is similar, we need to
give our API key which can be retrieved by signing up for an account on the
[OpenAI website](https://openai.com/api/) (see top-right of page). We then pass the API key below:

In [9]:
import os

# os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY' # SRA_DEBUGGING: Uncomment when done testing.

If using OpenAI via Azure you should also set:

```python
os.environ['OPENAI_API_TYPE'] = 'azure'
# API version to use (Azure has several)
os.environ['OPENAI_API_VERSION'] = '2022-12-01'
# base URL for your Azure OpenAI resource
os.environ['OPENAI_API_BASE'] = 'https://your-resource-name.openai.azure.com'
```

Then we decide on which model we'd like to use, there are several options but we will go with `text-davinci-003`:

In [10]:
from langchain_openai import ChatOpenAI

# Initialize with a modern model
turbo = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # or "gpt-4" or other available models
    temperature=0.7
)

Alternatively if using Azure OpenAI we do:

```python
from langchain_openai import AzureOpenAI

llm = AzureOpenAI(
    deployment_name="your-azure-deployment", 
    model_name="gpt-3.5-turbo"
)
```

We'll use the same simple question-answer prompt template as before with the Hugging Face example. The only change is that we now pass our OpenAI LLM `turbo`:

In [11]:
llm_chain = LLMChain(
    prompt=prompt,
    llm=turbo
)

print(llm_chain.run(question))

C:\Users\Siraj\AppData\Local\Temp\ipykernel_48220\2687700235.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
C:\Users\Siraj\AppData\Local\Temp\ipykernel_48220\2687700235.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm_chain.run(question))


A blade of grass does not have any eyes. Grass does not have sensory organs like eyes.


The same works again for multiple questions using `generate`:

In [15]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
llm_result = llm_chain.generate(qs)


In [18]:
res = [generation[0].message.content for generation in llm_result.generations]

for question, response in zip(qs, res):
    print("="*100)
    print(f"QUESTION: {question}")
    print(f"RESPONSE: {response}")
    print("="*100 + "\n")

QUESTION: {'question': 'Which NFL team won the Super Bowl in the 2010 season?'}
RESPONSE: The Green Bay Packers won Super Bowl XLV in the 2010 season.

QUESTION: {'question': 'If I am 6 ft 4 inches, how tall am I in centimeters?'}
RESPONSE: You are 193 centimeters tall.

QUESTION: {'question': 'Who was the 12th person on the moon?'}
RESPONSE: Harrison Schmitt was the 12th person to walk on the moon during the Apollo 17 mission in December 1972.

QUESTION: {'question': 'How many eyes does a blade of grass have?'}
RESPONSE: A blade of grass does not have any eyes. Grass plants do not have the ability to see as they do not have sensory organs like eyes.



Note that the below format doesn't feed the questions in iteratively but instead all in one chunk.

In [20]:
qs = [
    "Which NFL team won the Super Bowl in the 2010 season?",
    "If I am 6 ft 4 inches, how tall am I in centimeters?",
    "Who was the 12th person on the moon?",
    "How many eyes does a blade of grass have?"
]
res = llm_chain.batch(qs)

In [22]:
for item in res:
    print("="*100)
    print(f"QUESTION: {item['question']}")
    print(f"RESPONSE: {item['text']}")
    print("="*100 + "\n")

QUESTION: Which NFL team won the Super Bowl in the 2010 season?
RESPONSE: The Green Bay Packers won Super Bowl XLV in the 2010 season.

QUESTION: If I am 6 ft 4 inches, how tall am I in centimeters?
RESPONSE: You are 193.04 centimeters tall.

QUESTION: Who was the 12th person on the moon?
RESPONSE: Harrison Schmitt

QUESTION: How many eyes does a blade of grass have?
RESPONSE: A blade of grass does not have any eyes. Grass does not have visual organs like eyes.



Now we can try to answer all question in one go, as mentioned, more powerful LLMs like `gpt-3.5-turbo` will be more likely to handle these more complex queries.

In [23]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=turbo
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

print(llm_chain.run(qs_str))

1. The Green Bay Packers won the Super Bowl in the 2010 season.
2. You would be approximately 193 centimeters tall.
3. The 12th person on the moon was astronaut Charles "Pete" Conrad.
4. A blade of grass does not have eyes.


---